In [ ]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('/home/graduates/cyn/Cell2Map')
from Cell2Map import common
import matplotlib.pyplot as plt
import scanpy as sc

from Cell2Map import map_utils
from Cell2Map import autoencoder
from Cell2Map import utils
import anndata as ad

In [3]:
## Noise level 5%, mean of 5 cells per spot
## To use different simulation data, just change the data path
scRNA_path="../data/Hippocampus/scRNA/scRNA_Noise05.csv"
cell_type_path="../data/Hippocampus/scRNA/cell_labels.csv"
st_path="../data/Hippocampus/5/ST_expression.csv"
coordinates_path="../data/Hippocampus/5/Coordinates.csv"
ground_truth_path="../data/Hippocampus/5/ground_truth.csv"
ground_truth = common.read_file(ground_truth_path)
output_folder="../result/Hippocampus/xiaorong/distance_term"
mean_cell_numbers=5

In [ ]:
celltype_col='CellType'

# Read the data from the specified paths
# sc_adata: single-cell RNA-seq data
# spatial_adata: spatial transcriptomics data
# cell_type_number_eachspot_data: estimated number of each cell type in each spot
# coordinates_data: coordinates of each spot
sc_adata, spatial_adata, coordinates_data =\
        common.read_data(scRNA_path, cell_type_path, st_path, coordinates_path,celltype_col)

# Extract the cell type information from the single-cell data
# Rename the index and the 'CellType' column for clarity
cell_type=pd.DataFrame(sc_adata.obs['CellType'])
cell_type.index = [str(idx)[5:] for idx in cell_type.index]
cell_type['CellType'] = [str(idx)[5:] for idx in cell_type['CellType']]

# estimate the number of cells in each spot
cell_number_to_node_assignment=common.estimate_cell_number_RNA_reads(spatial_adata.to_df().T, int(mean_cell_numbers))

In [ ]:
sc_adata,spatial_adata=map_utils.process_adatas(sc_adata,spatial_adata)

/home/graduates/anaconda3/envs/celloc/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [ ]:
sc_adata

AnnData object with n_obs × n_vars = 23372 × 20419
    obs: 'CellType'
    var: 'n_cells'
    uns: 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

In [ ]:
spatial_adata

AnnData object with n_obs × n_vars = 4656 × 20419
    obs: 'rna_count_based_density'
    var: 'n_cells'
    uns: 'log1p'
    obsm: 'spatial'

In [ ]:
sc_adata,spatial_adata=autoencoder.embedding_feature(sc_adata,spatial_adata)

The graph contains 196250 edges, 23372 cells.
8.396799589252096 neighbors per cell on average.
The graph contains 163805 edges, 4656 cells.
35.181486254295535 neighbors per cell on average.


100%|██████████| 500/500 [00:55<00:00,  8.94it/s]


In [ ]:
Deep_map=map_utils.map_cell_to_space(sc_adata,spatial_adata,b_init=cell_number_to_node_assignment,device='cuda:0')

/home/graduates/cyn/DEEPIAC/DEEPIAC/map_optimizer.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.d=torch.tensor(d,device=device, dtype=torch.float32)


seed:1000
Total_loss: 20.757, gv_term: 0.903, vg_term: 0.490, density_term: 8.446, expression_term: 13.704, 
Total_loss: 20.722, gv_term: 0.917, vg_term: 0.490, density_term: 8.446, expression_term: 13.683, 
Total_loss: 20.695, gv_term: 0.923, vg_term: 0.490, density_term: 8.447, expression_term: 13.661, 
Total_loss: 20.671, gv_term: 0.926, vg_term: 0.490, density_term: 8.448, expression_term: 13.640, 
Total_loss: 20.649, gv_term: 0.928, vg_term: 0.490, density_term: 8.449, expression_term: 13.618, 
Total_loss: 20.627, gv_term: 0.930, vg_term: 0.490, density_term: 8.451, expression_term: 13.596, 
Total_loss: 20.606, gv_term: 0.931, vg_term: 0.490, density_term: 8.453, expression_term: 13.574, 
Total_loss: 20.586, gv_term: 0.931, vg_term: 0.490, density_term: 8.455, expression_term: 13.552, 
Total_loss: 20.566, gv_term: 0.930, vg_term: 0.490, density_term: 8.457, expression_term: 13.530, 
Total_loss: 20.547, gv_term: 0.930, vg_term: 0.490, density_term: 8.460, expression_term: 13.507, 


In [ ]:
Deep_map

AnnData object with n_obs × n_vars = 23372 × 4656
    obs: 'CellType'
    var: 'rna_count_based_density'

In [ ]:
sc_sp_map_df = pd.DataFrame(Deep_map.X, index=sc_adata.obs_names, columns=spatial_adata.obs_names)


In [ ]:
# Remove the first 5 characters from the column and index names of the DataFrame
sc_sp_map_df.columns = [str(col)[5:] for col in sc_sp_map_df.columns]
sc_sp_map_df.index = [str(idx)[5:] for idx in sc_sp_map_df.index]

# Initialize an empty dictionary to store the results
result_dict = {}

# For each column in the DataFrame, find the cell names with the highest values
# The number of cell names to find is determined by cell_number_to_node_assignment
# Store the results in the result_dict dictionary
for i, column in enumerate(sc_sp_map_df.columns.values):
    result_dict[column] = sc_sp_map_df[column].nlargest(cell_number_to_node_assignment[i]).index

# Convert the result_dict dictionary to a DataFrame
# The 'Predict' column contains the spot names, and the 'Values' column contains the cell names
# The 'Values' column is exploded so that each cell name has its own row
max_column_names = pd.DataFrame({'Predict': list(result_dict.keys()), 'Values': [list(index_obj) for index_obj in result_dict.values()]}).explode('Values')

# Convert the cell names in the 'Values' column to strings
max_column_names['Values'] = max_column_names['Values'].astype(str)

# Set the 'Values' column as the index of the DataFrame
max_column_names = max_column_names.set_index('Values')
max_column_names=max_column_names.assign(Lable='S0')
max_column_names=max_column_names.assign(CellType='type')

# Remove rows where the index is "nan"
max_column_names = max_column_names[max_column_names.index!="nan"]

# For each row in the DataFrame, replace 'type' in the 'CellType' column with the actual cell type from the cell_type DataFrame
for i,cell_id in enumerate(max_column_names.index.values):
    max_column_names.iloc[i,1]=ground_truth.loc[cell_id][0]
    max_column_names.iloc[i,2]=ground_truth.loc[cell_id][1]

# Reindex the coordinates_data DataFrame based on the 'Predict' column of the max_column_names DataFrame
# This gives the predicted coordinates for each cell
predict_coordinates=coordinates_data.reindex(max_column_names['Predict'])

# Add the predicted x and y coordinates to the max_column_names DataFrame
max_column_names['predict_x']=predict_coordinates['row'].values
max_column_names['predict_y']=predict_coordinates['col'].values

max_column_names

/tmp/ipykernel_1698436/3400311695.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_column_names.iloc[i,1]=ground_truth.loc[cell_id][0]
/tmp/ipykernel_1698436/3400311695.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_column_names.iloc[i,2]=ground_truth.loc[cell_id][1]


,Predict,Lable,CellType,predict_x,predict_y
Values,,,,,
V10882,S1,S1,Microglia,2,33
V552,S2,S2,Cajal Retzius,3,7
V864,S3,S3,Polydendrocyte,3,10
V21370,S3,S3,Entorhinal,3,10
V17277,S3,S3843,Polydendrocyte,3,10
...,...,...,...,...,...
V17357,S4655,S4655,Polydendrocyte,90,29
V15965,S4656,S4656,Entorhinal,90,31
V709,S4656,S4656,Cajal Retzius,90,31


In [ ]:
assigned_locations_path = os.path.join(output_folder, "2")
os.makedirs(assigned_locations_path, exist_ok=True)
result_csv_path=os.path.join(assigned_locations_path,"result.csv")
max_column_names.to_csv(result_csv_path)
scRNA_path=os.path.join(assigned_locations_path,"sc_adata.h5ad")
sc_adata.write_h5ad(scRNA_path)
spatial_path=os.path.join(assigned_locations_path,"sp_adata.h5ad")
spatial_adata.write_h5ad(spatial_path)
deep_map_path=os.path.join(assigned_locations_path,"deep_map.h5ad")
Deep_map.write_h5ad(deep_map_path)

